Author: Ronny F. Pacheco Date: Sep 2024
Copyright: © 2024 Ronny Pacheco License: MIT License

---

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Needed modules

In [6]:
# Import need packages
import pickle
import pandas as pd
import json
import os

In [2]:
## https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore

@register_cell_magic
def skip(line, cell):
    return

# Pickle save

In [3]:
%%skip
# =============================================================================
# main function
# =============================================================================
def data_save_load(option):
    """
    This function is used to save or load data for the Jupyter notebook.
    
    Parameters:
    option (str): Either 'save' or 'load' to save or load variables.
    
    Returns:
    dict: Dictionary of loaded variables (only when option is 'load').
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    path_file = os.path.join(path_folder, "variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as file:
            variables_dict = {}  # Dictionary to save the variables
            pickle.dump(variables_dict, file)
    elif option == "load":
        with open(path_file, "rb") as file:
            variables = pickle.load(file)
        # Now load the variables
        for var_key, var_value in variables.items():
            variables[var_key] = var_value

# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="save")

# Prepare data

In [4]:
# Load the JSON dict into a Python dictionary
path_json_file = "./data/neg_gtf_relation.json"

with open(path_json_file, "r") as f:
    dict_neg_gtf = json.load(f)

In [5]:
# Check the size of the dictionary
print(f"Size of the dictionary: {len(dict_neg_gtf)} elements")

Size of the dictionary: 681 elements


# 2. Create table

In [7]:
def gene_finder(sublist):
    """
    This function is used to find the gene name from the sublist without repetition
    """
    gene_name = ""
    gene_strand = ""
    if len(sublist) == 0:
        gene_name = "intergenic"
        gene_strand = None
    else:
        gene_list = []
        gene_strand = []
        for elem in sublist:
            if elem.get("gene_id") is not None:
                gene_list.append(elem.get("gene_id"))
                gene_strand.append(elem.get("strand"))
        gene_list = list(set(gene_list))  # Remove duplicates
        gene_name = ", ".join(gene_list)
        gene_strand = list(set(gene_strand))
        gene_strand = ", ".join(gene_strand)
    return gene_name, gene_strand

def transcript_finder(sublist):
    """
    This function is used to find the transcript name from the sublist without repetition
    """
    transcript_dict = {}
    transcript = False
    cds = False
    utr3 = False
    utr5 = False
    for elem in sublist:
        # Take transcript data
        if elem.get("transcript_id") is not None:
            # Create a key with the transcript_id if it does not exist already
            if transcript_dict.get(elem.get("transcript_id")) is None:
                transcript_dict[elem.get("transcript_id")] = []
            transcript = True
        # Now let's go for the CDS, 3utr and 5utr
        if elem.get("CDS") == True:
            transcript_dict[elem.get("parent_id")].append("CDS")
            cds = True
        if elem.get("3utr") == True:
            transcript_dict[elem.get("parent_id")].append("3utr")
            utr3 = True
        if elem.get("5utr") == True:
            transcript_dict[elem.get("parent_id")].append("5utr")
            utr5 = True
    
    return transcript_dict, transcript, cds, utr3, utr5

    

In [8]:
# Let's create a table where one column is the "neg name" and the other the "gene_name".
# If there is no gene_name, its intergenic region, so we will put "intergenic" in the gene_name column
list_data = []
for key, value in dict_neg_gtf.items():
    neg_name = key
    
    # Find the gen_id for each neg name
    gene_name, gene_strand = gene_finder(value)
    # list_data.append([neg_name, gene_name])

    # Find the transcript_id for each neg name
    transcript_name = ""
    transcript, cds, utr3, utr5 = False, False, False, False
    if gene_name != "intergenic":
        transcript_name, transcript, cds, utr3, utr5 = transcript_finder(value)
    else:
        transcript_name = "intergenic"
    list_data.append([neg_name, gene_name, gene_strand, transcript_name, transcript, cds, utr3, utr5])

# Create a pandas dataframe
df_neg_gtf = pd.DataFrame(list_data, columns=["neg_name", "gene_name", "gene_strand", "transcript_name", "transcript", "cds", "utr3", "utr5"])

In [9]:
print(df_neg_gtf.shape)
print(df_neg_gtf.dtypes)
df_neg_gtf.head()

(681, 8)
neg_name           object
gene_name          object
gene_strand        object
transcript_name    object
transcript           bool
cds                  bool
utr3                 bool
utr5                 bool
dtype: object


,neg_name,gene_name,gene_strand,transcript_name,transcript,cds,utr3,utr5
0,rejected_noCDS_c01.10,LINF_010006300,-,{'LINF_01T0006300': ['3utr']},True,False,True,False
1,rejected_noCDS_c01.20,LINF_010009600,+,{'LINF_01T0009600': ['5utr']},True,False,False,True
2,rejected_noCDS_c01.30,LINF_010010350,+,{'LINF_01T0010350': []},True,False,False,False
3,rejected_noCDS_c01.40,LINF_010013350,+,{'LINF_01T0013350': []},True,False,False,False
4,rejected_noCDS_c01.50,intergenic,None,intergenic,False,False,False,False


# Merge with the csv data

In [10]:
# Load data
og_data = pd.read_csv("../22.GFF_Creation/data_named/negative_database_nomatch_corrected_named.csv", sep=",", header=0)
print(og_data.shape)
print(og_data.dtypes)
og_data.head()

(682, 6)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
name       object
dtype: object


,sseqid,sstart,send,sstrand,sseq,name
0,LinJ.01,36103,36242,plus,AGACAGACCGACACACGCAGCCGTGTGATGCCGCCGCCGAGGGCAG...,rejected_noCDS_c01.10
1,LinJ.01,113760,114388,plus,CAGCGCCATGCACGACATGGCCGCTGACGTCCGTAGCCCTAACTCG...,rejected_noCDS_c01.20
2,LinJ.01,146412,146530,plus,GCGAATTGTGTTCTGCGCATGCCTCTTCTCTGCCGTGCAGCATGCG...,rejected_noCDS_c01.30
3,LinJ.01,261866,262439,plus,CGGACTTGGCAAGTGGCCGCCATCGATGAAAACGCACCATGCCTTT...,rejected_noCDS_c01.40
4,LinJ.01,271363,271650,plus,CGAACGCCGCCCTCAATCGCGCGCTGAACTTCACGCGGCGGTCGAC...,rejected_noCDS_c01.50


In [11]:
# change the colum name "new_IDs" to "neg_name"
og_data.rename(columns={"name": "neg_name"}, inplace=True)

# Remove the "sstrand" column
og_data.drop(columns=["sstrand"], inplace=True)
og_data.head()

,sseqid,sstart,send,sseq,neg_name
0,LinJ.01,36103,36242,AGACAGACCGACACACGCAGCCGTGTGATGCCGCCGCCGAGGGCAG...,rejected_noCDS_c01.10
1,LinJ.01,113760,114388,CAGCGCCATGCACGACATGGCCGCTGACGTCCGTAGCCCTAACTCG...,rejected_noCDS_c01.20
2,LinJ.01,146412,146530,GCGAATTGTGTTCTGCGCATGCCTCTTCTCTGCCGTGCAGCATGCG...,rejected_noCDS_c01.30
3,LinJ.01,261866,262439,CGGACTTGGCAAGTGGCCGCCATCGATGAAAACGCACCATGCCTTT...,rejected_noCDS_c01.40
4,LinJ.01,271363,271650,CGAACGCCGCCCTCAATCGCGCGCTGAACTTCACGCGGCGGTCGAC...,rejected_noCDS_c01.50


In [12]:
# Merge the two dataframes by the "neg_name" column
df_merged = pd.merge(df_neg_gtf, og_data, on="neg_name", how="inner")
print(df_merged.shape)
print(df_merged.dtypes)
df_merged.head()

(682, 12)
neg_name           object
gene_name          object
gene_strand        object
transcript_name    object
transcript           bool
cds                  bool
utr3                 bool
utr5                 bool
sseqid             object
sstart              int64
send                int64
sseq               object
dtype: object


,neg_name,gene_name,gene_strand,transcript_name,transcript,cds,utr3,utr5,sseqid,sstart,send,sseq
0,rejected_noCDS_c01.10,LINF_010006300,-,{'LINF_01T0006300': ['3utr']},True,False,True,False,LinJ.01,36103,36242,AGACAGACCGACACACGCAGCCGTGTGATGCCGCCGCCGAGGGCAG...
1,rejected_noCDS_c01.20,LINF_010009600,+,{'LINF_01T0009600': ['5utr']},True,False,False,True,LinJ.01,113760,114388,CAGCGCCATGCACGACATGGCCGCTGACGTCCGTAGCCCTAACTCG...
2,rejected_noCDS_c01.30,LINF_010010350,+,{'LINF_01T0010350': []},True,False,False,False,LinJ.01,146412,146530,GCGAATTGTGTTCTGCGCATGCCTCTTCTCTGCCGTGCAGCATGCG...
3,rejected_noCDS_c01.40,LINF_010013350,+,{'LINF_01T0013350': []},True,False,False,False,LinJ.01,261866,262439,CGGACTTGGCAAGTGGCCGCCATCGATGAAAACGCACCATGCCTTT...
4,rejected_noCDS_c01.50,intergenic,None,intergenic,False,False,False,False,LinJ.01,271363,271650,CGAACGCCGCCCTCAATCGCGCGCTGAACTTCACGCGGCGGTCGAC...


In [13]:
df_merged.rename(columns={
    "sseqid" : "neg_chrom",
    "sstart" : "neg_start",
    "send" : "neg_end",
    "sseq" : "neg_seq"
}, inplace=True)

df_merged.head()

,neg_name,gene_name,gene_strand,transcript_name,transcript,cds,utr3,utr5,neg_chrom,neg_start,neg_end,neg_seq
0,rejected_noCDS_c01.10,LINF_010006300,-,{'LINF_01T0006300': ['3utr']},True,False,True,False,LinJ.01,36103,36242,AGACAGACCGACACACGCAGCCGTGTGATGCCGCCGCCGAGGGCAG...
1,rejected_noCDS_c01.20,LINF_010009600,+,{'LINF_01T0009600': ['5utr']},True,False,False,True,LinJ.01,113760,114388,CAGCGCCATGCACGACATGGCCGCTGACGTCCGTAGCCCTAACTCG...
2,rejected_noCDS_c01.30,LINF_010010350,+,{'LINF_01T0010350': []},True,False,False,False,LinJ.01,146412,146530,GCGAATTGTGTTCTGCGCATGCCTCTTCTCTGCCGTGCAGCATGCG...
3,rejected_noCDS_c01.40,LINF_010013350,+,{'LINF_01T0013350': []},True,False,False,False,LinJ.01,261866,262439,CGGACTTGGCAAGTGGCCGCCATCGATGAAAACGCACCATGCCTTT...
4,rejected_noCDS_c01.50,intergenic,None,intergenic,False,False,False,False,LinJ.01,271363,271650,CGAACGCCGCCCTCAATCGCGCGCTGAACTTCACGCGGCGGTCGAC...


In [14]:
# Export to CSV and Excel file
path_csv = "./data/neg_relationship.csv"
path_excel = "./data/neg_relationship.xlsx"

# Export to CSV
df_merged.to_csv(path_csv, index=False, sep=",", header=True)

# Export to Excel
df_merged.to_excel(path_excel, index=False, header=True)